# Gemini API intro

In [ ]:
from google import genai

# looks automatically after the key
# one of GOOGLE_API_KEY and GEMINI_API_KEY
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI learns patterns from data to make decisions and perform tasks.


In [2]:

def ask_gemini(prompt, model = "gemini-2.5-flash"):
    response = client.models.generate_content(
        model=model,
        contents=prompt,
    )

    return response

response = ask_gemini("Give me 5 some data engineering jokes, structure it in short points")
print(response.text)

Here are 5 data engineering jokes, structured in short points:

1.  A data engineer's job is 10% coding, 90% debugging why the data from the 'clean' source looks like it was generated by a toddler with a keyboard.
2.  What's a data engineer's biggest fear? A fully automated data pipeline... that actually works for more than a day.
3.  Heard a data engineer say they finished their project. I almost choked on my coffee – that's like saying you've finished eating a never-ending buffet.
4.  A data scientist walks into a bar and asks for a perfectly clean, curated dataset. The bartender says, "Sorry, we only serve real-world data here."
5.  My data pipeline isn't broken, it's just... *exercising its right to unpredictability*.


In [ ]:
from pydantic import BaseModel

# knows that GenerateContentResponse is a pydantic model
# -> we can work with it in a OOP manner
isinstance(response, BaseModel)

True

In [10]:
dict(response).keys()

dict_keys(['sdk_http_response', 'candidates', 'create_time', 'model_version', 'prompt_feedback', 'response_id', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])

In [12]:
response.model_version

'gemini-2.5-flash'

In [14]:
response.sdk_http_response

HttpResponse(
  headers=<dict len=11>
)

In [16]:
response.candidates

[Candidate(
   content=Content(
     parts=[
       Part(
         text="""Here are 5 data engineering jokes, structured in short points:
 
 1.  A data engineer's job is 10% coding, 90% debugging why the data from the 'clean' source looks like it was generated by a toddler with a keyboard.
 2.  What's a data engineer's biggest fear? A fully automated data pipeline... that actually works for more than a day.
 3.  Heard a data engineer say they finished their project. I almost choked on my coffee – that's like saying you've finished eating a never-ending buffet.
 4.  A data scientist walks into a bar and asks for a perfectly clean, curated dataset. The bartender says, "Sorry, we only serve real-world data here."
 5.  My data pipeline isn't broken, it's just... *exercising its right to unpredictability*."""
       ),
     ],
     role='model'
   ),
   finish_reason=<FinishReason.STOP: 'STOP'>,
   index=0
 )]

## Tokens

- basic unit of text for LLMs
- can be as short as one character or as long as one word

- tokens used for billing

Gemini free tier 
- Requests per minute (RPM): 10
- Tokens per minute (TPM): 250 000
- Requests per day (RPD): 250

In [ ]:
# thinking is expensive
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=187,
  prompt_token_count=15,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=15
    ),
  ],
  thoughts_token_count=1197,
  total_token_count=1399
)

## Thinking

- hyperparameter to allocate more compute for complex tasks

In [21]:
from google.genai import types

prompt = "Give me 5 some data engineering jokes, structure it in short points"

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)

print(response.text)


Here are 5 data engineering jokes, structured in short points:

1.  **A data engineer walks into a bar.**
    *   Orders a beer.
    *   Bartender asks, "Want to denormalize that?"

2.  **Why did the ELT job get fired?**
    *   It kept asking "Why transform now?"

3.  **What's a data engineer's favorite type of music?**
    *   Pipelines, especially when they're flowing!

4.  **A data scientist, a data analyst, and a data engineer are on a plane.**
    *   The plane crashes.
    *   Who survives? The data engineer, because they built the emergency parachute system that everyone else just *assumed* was working.

5.  **Heard about the data engineer who tried to automate everything?**
    *   His morning coffee ran through a DAG.
    *   But the downstream tasks kept failing because the beans weren't idempotently roasted.


In [22]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=219,
  prompt_token_count=15,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=15
    ),
  ],
  total_token_count=234
)

## System instruction

- hyperparameter to guide model behavior

In [29]:
system_instruction = """
You are an expert in python programming, you will always provide idiomatic code, i.e.
pythonic code. So when you see my code or my question, be very critical, but answer
in a SHORT and CONCISE way. Also be constructive to help me improve. 
"""

prompt = """
Explain OOP and dunder methods.
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_instruction
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)

print(response.text)



**OOP (Object-Oriented Programming):**

*   Organizes code around "objects" (data + behavior).
*   Key principles:
    *   **Encapsulation:** Bundling data and methods.
    *   **Inheritance:** Creating new classes from existing ones.
    *   **Polymorphism:** Objects taking on many forms.
    *   **Abstraction:** Hiding complex implementation.

**Dunder (Double Underscore) Methods:**

*   Special methods with double underscores (e.g., `__init__`, `__str__`).
*   Used for operator overloading, object initialization, string representation, etc.
*   Allow you to customize how objects behave in various situations (e.g., when added, printed).



In [31]:
metadata = response.usage_metadata
metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=158,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=158
    ),
  ],
  prompt_token_count=70,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=70
    ),
  ],
  total_token_count=228
)

In [ ]:
print(f"{metadata.candidates_token_count = }") # output
print(f"{metadata.prompt_token_count = }") # prompt + system instruction
print(f"{metadata.total_token_count = }")

metadata.candidates_token_count = 158
metadata.prompt_token_count = 70
metadata.total_token_count = 228


In [ ]:

len(prompt.split()), len(system_instruction.split())

(5, 43)

## Temperature

- controls randomness of output -> 'creative'

its a hyperparameter that can be adjusted to influence the diversity and creativity of the generated text.

In [40]:
story = "write a 2 sentence story about a gray rabbit"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=0
        # system_instruction=system_instruction
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)

print(response.text)

The gray rabbit twitched its nose, sensing the distant rumble of a lawnmower and immediately darted into the overgrown rose bushes, its fluffy tail disappearing in a flash of white. Safe within the thorny embrace, it nibbled on a fallen petal, the sweet scent masking the mechanical threat.



In [41]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=0
        # system_instruction=system_instruction
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)

print(response.text)

The gray rabbit twitched its nose, sensing the distant rumble of a lawnmower and immediately darted into the overgrown rose bushes. Safe within the thorny embrace, it nibbled on a fallen petal, the sweet scent masking the fear that still lingered in its twitching whiskers.



In [42]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=2.0
        # system_instruction=system_instruction
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)

print(response.text)

The little gray rabbit twitched its nose, scenting danger in the wind, and darted under the thorny bushes, disappearing into the shadowed world of the undergrowth. He knew the fox was out hunting tonight.



In [45]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=2.0
        # system_instruction=system_instruction
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)

print(response.text)

The little gray rabbit hopped through the tall grass, his nose twitching as he searched for a juicy clover. Suddenly, a shadow loomed overhead, and he knew his playful exploration had led him too close to the hawk's hunting ground. 



## Multimodal input

input text and image

In [48]:
text_input = "Describe this image shortly"
image_input = {"mime_type": "image/png", "data": open("bella.png", 'rb').read()}


response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=dict(
        parts=[dict(text = text_input), dict(inline_data = image_input)]
    )
)

print(response.text)

This is a close-up photo of a fluffy, gray rabbit wearing a miniature plush Swedish graduation cap (studentmössa) with a blue and yellow ribbon draped over its back. The rabbit is resting on a gray carpet.
